<a href="https://colab.research.google.com/github/nourelhouda03/text-classification-models/blob/main/ara_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install  alphabet_detector 
!pip install pyarabic
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
# Load the dataset into a pandas dataframe.
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/datasetfinal.xlsx")

from alphabet_detector import AlphabetDetector
import re
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem.isri import ISRIStemmer

#remove any othe language letters
def keep_only_arabic(text):
    ad = AlphabetDetector()
    clean_lines = list()
    for line in text.splitlines():
        clean_line = list()
        for word in line.split():
            if len(word) > 1:
                if ad.is_arabic(word):
                    if word.isalpha():
                        clean_line.append(word)
        clean_lines.append(' '.join(clean_line))
    return '\n'.join(clean_lines)

# remove empty lines
def remove_empty_lines(text):
    lines = [s.rstrip() for s in text.split("\n") if s.rstrip()]
    return '\n'.join(lines)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
import pyarabic.araby as araby
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1", text) # Remove longation
    return araby.strip_tashkeel(text)
    
#delete everything except text 
def char_rmvl(sentence):
  sen_split = sentence.split()
  for idx1,word in enumerate(sen_split):       
             sen_split[idx1] = re.sub('[^a-zا-ي]', '', word)       
  sentence = ' '.join(sen_split)    
  return sentence 
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح",
                  "أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","","أنتما","أنتن","أنتِ",
                  "أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا",
                  "إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن",
                  "إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة",
                  "ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي",
                  "الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي",
                  "الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام",
                  "ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن",
                  "بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل",
                  "تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا",
                  "حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون",
                  "دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي",
                  "راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع",
                  "شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا",
                  "عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ",
                  "عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما",
                  "كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم",
                  "كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن",
                  "لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا",
                  "لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال",
                  "مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن",
                  "من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا",
                  "هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء",
                  "هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت",
                  "واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو",
                  "وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان","أنتم" , 'إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 
                  'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 
                    'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى',
                    'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن',
                    'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا',
                    'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 
                    'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 
                    'فيها', 'قد', 'كأن', 'كأنما', 'كأي', 'كأين', 'كذا', 'كذلك', 'كل', 'كلا', 'كلاهما', 'كلتا', 'كلما', 'كليكما', 'كليهما', 'كم', 'كم', 'كما', 'كي', 'كيت', 'كيف', 'كيفما', 'لا',
                    'لاسيما', 'لدى', 'لست', 'لستم', 'لستما', 'لستن', 'لسن', 'لسنا', 'لعل', 'لك', 'لكم', 'لكما', 'لكن', 'لكنما', 'لكي', 'لكيلا', 'لم', 'لما', 'لن', 'لنا', 'له', 'لها', 'لهم', 'لهما', 
                    'لهن', 'لو', 'لولا', 'لوما', 'لي', 'لئن', 'ليت', 'ليس', 'ليسا', 'ليست', 'ليستا', 'ليسوا', 'ما', 'ماذا', 'متى', 'مذ', 'مع', 'مما', 'ممن', 'من', 'منه', 'منها', 'منذ', 'مه', 'مهما',
                    'نحن', 'نحو', 'نعم', 'ها', 'هاتان', 'هاته', 'هاتي', 'هاتين', 'هاك', 'هاهنا', 'هذا', 'هذان', 'هذه', 'هذي', 'هذين', 'هكذا', 'هل', 'هلا', 'هم', 'هما', 'هن', 'هنا', 'هناك', 'هنالك',
                    'هو', 'هؤلاء', 'هي', 'هيا', 'هيت', 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن', 'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا', 'يمالى', 'قل', 'كثر', 'ألي', 'ليك', 'ليكم', 
                    'نتيا', 'نتوما', 'رانا' ,'بصح', 'هوما', 'وين', 'أمبعد', 'أومبعد', 'شوية', 'شويا', 'وش', 'واش', 'بوش', 'بواش','و'}
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])


def all_togther(text):
  text = keep_only_arabic(text)
  text = remove_empty_lines(text)
  text = remove_emoji(text)
  text = normalizeArabic(text)
  text = char_rmvl(text)
  #text = text.lower()
  text = remove_stop_words(text)
  text = ISRIStemmer().suf32(text)
  return text


df['comments'] = df['comments'].apply(lambda x:all_togther(x))
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["comments"], inplace=True)

Mounted at /content/drive
  Created wheel for alphabet-detector: filename=alphabet_detector-0.0.7-py3-none-any.whl size=2445 sha256=1e8839baea9133506d417d6209a379036cb883fba584b65d31815bd4d6e0f1de
  Stored in directory: /root/.cache/pip/wheels/22/8c/ab/4afb1765f2b8450f894a1f06c9aa2b3f8e73f2fb8b55849e17
Successfully built alphabet-detector
     |████████████████████████████████| 126 kB 6.9 MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 69.8 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 636 kB 68.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import tensorflow as tf
import numpy as np
#creating BERT tokenizer.
from transformers import BertTokenizer, TFBertModel, BertConfig

pretrained_weights = 'asafaya/bert-base-arabic'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

Comments = df['comments'].values
print(Comments)

#tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
#model = AutoModel.from_pretrained("asafaya/bert-base-arabic")


['مهرج صوفي قبوري جاهل يفسر يحب واتباعه اجهل'
 'اسال علي اسمك مهرج العصر الرسول صلي اله سلم مقالناش ديروا عيد ميلادي لانه عمل اليهود والنصاري انشا اله تحشر مع'
 'يالحيوان الدرس بدعه ياصوفي لماذا لاتقي اله اله ياخذك اخذت عزيز مقتدر يالجاهل'
 ... 'الصخيرات المغرب اجتماع اليبين عدو المستحيل يتكلم السيد كداب'
 'ونوض اكبر المنافقين اعطي التيساع الشيوعيه تقبل الصحرا مغربيه علي رغم انفك وانف الي النفاق والكذب علي الشعب وجهك اصبح اسود ادفع لابن بطوش حتي طيش اموال'
 'احشم راك كبير منافق الجزار ماتو رجال حكموها دواب تبون احشم علي سميتك']


In [4]:

input_ids = []
attention_masks = []

# For every sentence...
for sent in Comments:
    #print(sent)
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation = True,
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors='tf' ,          # Return pytorch tensors.    
                   ) 
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
print(len(input_ids))
print(len(attention_masks))
#input_ids and attention_masks arrays reshaping
input_ids = tf.convert_to_tensor(input_ids)
input_ids = np.reshape(input_ids, [9208,128])

attention_masks = tf.convert_to_tensor(attention_masks)
attention_masks = np.reshape(attention_masks, [9208,128])

# labels transformation (multi-class to binary)
labels = df['hate '].values
labels = [0 if y == 'no' else 1 for y in labels]

!pip install keras.utils

#Stratified Sampling (Train set = 70%, Validation set = 20%, Test set = 10%)
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(input_ids, labels, test_size=0.30, random_state=42, stratify=labels)
X_train_masks, X_eval_masks, _, _ = train_test_split(attention_masks, labels, test_size=0.30, random_state=42, stratify=labels)

X_val, X_test, y_val, y_test = train_test_split(X_eval, y_eval, test_size=0.3333, random_state=42, stratify=y_eval)
X_val_masks, X_test_masks, _, _ = train_test_split(X_eval_masks, y_eval, test_size=0.3333, random_state=42, stratify=y_eval)

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

print(X_train.shape[0])
print(X_eval.shape[0])
print(X_test.shape[0])
print(X_val.shape[0])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


9208
9208
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2657 sha256=49f70088be7f97a685e3b5e86eb2cba06e6956cb53ee465bd57fc8395ae67763
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils
6445
2763
921
1842


In [5]:

#Focal Loss Function for binary classification

import tensorflow as tf
from keras import backend as K

def focal_loss(gamma=2., alpha=.25):
	def focal_loss_fixed(y_true, y_pred):
		pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
		pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
		return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
	return focal_loss_fixed

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

BERT_Model = TFBertModel.from_pretrained("asafaya/bert-base-arabic")

BERT_Model.count_params()

def BERT_model():
    
    ##pre_trained BERT model
    BERT_Model = TFBertModel.from_pretrained("asafaya/bert-base-arabic")

    ## BERT Model Inputs
    input_ids = layers.Input(shape=(128,), dtype=tf.int32) 
    attention_mask = layers.Input(shape=(128,), dtype=tf.int32)

   
#BERT_Outputs[0]==> Tokens Representation
#BERT_Outputs[1]==> Sentence Representation ([CLS] Output)
    BERT_Outputs = BERT_Model(
        [input_ids,
        attention_mask]
    )
    
    outputs = layers.Dense(2, activation='softmax')(BERT_Outputs[1])

    model = keras.Model(
        inputs=[input_ids, attention_mask],
        outputs= [outputs]
    )
    return model

model1 = BERT_model()

model1.count_params()

optimizer = tf.optimizers.Adam(2e-5)

model1.compile(loss=['categorical_crossentropy'], optimizer=optimizer)

model1.compile(optimizer=optimizer, loss=[focal_loss(gamma=2., alpha=.25)])

model1.fit([X_train, X_train_masks], y_train, batch_size=32, epochs=4, validation_data=([X_val, X_val_masks], y_val))

outcomes = model1.predict([X_test, X_test_masks])



Some layers from the model checkpoint at asafaya/bert-base-arabic were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at asafaya/bert-base-arabic.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Some layers from the model checkpoint at asafaya/bert-base-arabic were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initial

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [6]:

import numpy as np
from sklearn.metrics import accuracy_score

y_pred =  [np.argmax(y, axis=None, out=None) for y in outcomes]
y_test_2 = [np.argmax(y, axis=None, out=None) for y in y_test] 

from sklearn.metrics import confusion_matrix,  classification_report, recall_score, precision_score

macro_recall = recall_score(y_test_2, y_pred, average='macro')
macro_precision = precision_score(y_test_2, y_pred, average='macro')
macro_f1_measure = 2*macro_recall*macro_precision/(macro_recall+macro_precision)
accuracy=accuracy_score(y_test_2, y_pred)
print('macro recall:')
print(macro_recall)

print('macro precision:')
print(macro_precision)

print('macro f1_measure:')
print(macro_f1_measure)

print("accuracy")
print(accuracy)




macro recall:
0.7267779806690312
macro precision:
0.7247027897503908
macro f1_measure:
0.7257389017531483
accuracy
0.7198697068403909
